In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import linalg
import math
from neicio.readstation import readStation
from neicio.shake import ShakeGrid
from neicio.gmt import GMTGrid
import time
from openquake.hazardlib.imt import from_string
from openquake.hazardlib.site import Site, SiteCollection
from openquake.hazardlib.geo import Point
from openquake.hazardlib.correlation import JB2009CorrelationModel
from openquake.hazardlib.correlation import BaseCorrelationModel
from matplotlib import cm
from Correlation.setup import initialize
from Correlation.loop import main
from Correlation.realizations import realizations



In [3]:
# Variable of interest                                                                                                                                                                                                        
voi = 'PGA'

# Intensity Factor
intensity_factor = 0.9

# Get shakemap for desired variable, PGA, uncertainty grid and stationdata                                                                                                                                                    
# Selected Stations: Units in pctg                                                                                                                                                                                            
shakemap = ShakeGrid('/Inputs/grid.xml', variable = '%s' % voi)

# Uncertainty Data: Units in ln(pctg)                                                                                                                                                                                         
uncertainty = ShakeGrid('/Inputs/uncertainty.xml', variable= 'STD%s' % voi)

# Station Data: Units in pctg                                                                                                                                                                                                 
stationlist = '/Inputs/stationlist.xml'
stationdata = readStation(stationlist)

print np.size(stationdata['pga'])
# Sets up grid spacing, site collections, and other data values
# optional parameters: dm, dn, the grid discritization defaulting to 1
variables = initialize(shakemap, uncertainty, stationdata, 2, 2)
M = variables['M']
N = variables['N']
K = variables['K']

# Calculate the random array, stored for testing
rand = np.random.randn(M*N).flatten()



26
	Initialization Time: 0.109070062637


In [4]:
R = [5, 10, 15, 20, 40]

f = open('workfileNR26_uncorr.txt', 'w')

f.write('Northridge, %d stations, max_R = %d\n' %(np.size(stationdata['pga']), R[-1]))
s = str(np.size(R))
f.write(s+'\n')
s = str(M)
f.write(s+'\n')
s = str(N)
f.write(s+'\n')

for j in range(0, M*N):
    s  = str(rand[j])
    f.write(s+'\n')

data_NEW = np.zeros([M*N, np.size(R)])

for k in range(0,np.size(R)):
    out = main(variables, R[k], voi, rand, 0.9)
    data_NEW[:,k] = np.reshape(out['data_new'], [M*N])
    for j in range(0, M*N):
        s = str(data_NEW[j,k])
        f.write(s+'\n')
                       
    
f.close()


Finishing step: 0
Finishing step: 5000
Total Time 1.18953084946
Pre loop Time 0.0175609588623
Inner loop time 1.10508871078
Outer loop time 0.0105457305908
Finishing step: 0
Finishing step: 5000
Total Time 2.38664007187
Pre loop Time 0.000443935394287
Inner loop time 2.29619860649
Outer loop time 0.0312557220459
Finishing step: 0
Finishing step: 5000
Total Time 3.16400504112
Pre loop Time 0.000479936599731
Inner loop time 3.03448200226
Outer loop time 0.069776058197
Finishing step: 0
Finishing step: 5000
Total Time 5.21475601196
Pre loop Time 0.000327110290527
Inner loop time 5.02604436874
Outer loop time 0.128612279892
Finishing step: 0
Finishing step: 5000
Total Time 50.2310869694
Pre loop Time 0.000385999679565
Inner loop time 49.5810496807
Outer loop time 0.580630302429


In [5]:
g = open('workfileNR26full_uncorr.txt', 'w')
g.write('Northridge, %d stations \n' %(np.size(stationdata['pga'])))
s = str(variables['M'])
g.write(s+'\n')
s = str(variables['N'])
g.write(s+'\n')

# Using the collected data, calculate correlation matrix
cor_model = BaseCorrelationModel
JB_cor_model = JB2009CorrelationModel(cor_model)
comb_cor_model = JB_cor_model._get_correlation_matrix(variables['site_collection_both'], from_string(voi))

# From Characterization of Uncertainty in Shakemaps, we have
# Sig11 = station_correlation_model
# Sig22 = grid_correlation_model
# Sig12 = Sig21^T

Sig11 = comb_cor_model[0:K, 0:K]
Sig12 = comb_cor_model[0:K, K:M*N+K]
Sig22 = comb_cor_model[K:M*N+K, K:M*N+K]

Sig11inv = linalg.pinv(Sig11)

# Xobs residuals
x = np.zeros([K,1])

# Calculate mean and covariance matrices specified in Jayaram and Baker 2009
mean = np.array(Sig12.T * Sig11inv * x)
cov = Sig22 - (Sig12.T*Sig11inv*Sig12)

# Find distribution 
R = np.linalg.cholesky(cov)
rand = np.reshape(rand, [M*N,1])
X = mean + R*rand

COR = np.reshape(X, [M,N])

#Multiply by uncertainty
X = np.multiply(COR, variables['uncertaintydata'])

DATA_NEW = np.multiply(variables['data'],np.exp(X))
for i in range(0,M):
    for j in range(0,N):

        s = str(DATA_NEW[i,j])
        g.write(s+' ')
g.close()